In [16]:
import numpy as np
import pandas as pd

In [17]:
"""
Inputs de muestra
"""

nombres = "Juan Prueba"

fecha_accidente = "2020-09"

sueldos = [
            58000,
            58000,
            68000,
            70000,
            65000,
            65000,
            65000,
            65000,
            65000,
            75000,
            70000,
            70000,           
]
sueldos = list(filter(None, sueldos))
sueldos = list(map(float,sueldos))

edad = 28

incapacidad = 33

tipo_accidente = 1 # 'in_itinere=0','en_ocasion=1'

In [18]:
"""
Preparación del dataframe general de ripte
"""

ripte = pd.read_csv('https://github.com/Demian762/Calculadora_RIPTE/blob/main/ripte.csv?raw=true')
ripte['fechas'] = ripte['fechas'].astype('datetime64')
ripte['fechas'] = ripte['fechas'].dt.to_period('M')

In [19]:
"""
Cargando las tasas de una pagina random porque al BNA le chupa un huevo
"""
tasas = pd.read_csv('https://github.com/Demian762/Calculadora_RIPTE/blob/main/tasas.csv?raw=true')
tasas['fechas'] = tasas['fechas'].astype('datetime64')
tasas['fechas'] = tasas['fechas'].dt.to_period('M')

In [20]:
"""
Creacion y merge del dataframe particular del cliente
"""

cliente = pd.DataFrame(sueldos)
cliente = cliente.rename(columns={0:'sueldos'})

cliente['fechas'] = pd.date_range(end=fecha_accidente,freq='M',periods=len(sueldos))
cliente['fechas'] = cliente['fechas'].dt.to_period('M')

cliente = pd.merge(cliente, ripte, on='fechas', how='left')
cliente = pd.merge(cliente, tasas, on="fechas", how="left")

indice = ripte[ripte['fechas']==fecha_accidente].final
indice = float(indice)

cliente['coeficiente'] = 0.0

cliente = cliente.assign(coeficiente = lambda x: (indice/x['final']))
cliente = cliente.assign(actualizados = lambda x: (x['sueldos']*x['coeficiente']))
cliente = cliente.round(2)

cliente.loc[0,"nombres"] = nombres
cliente.loc[0,"edad"] = edad
cliente.loc[0,"incapacidad"] = incapacidad
cliente.loc[0,"fecha"] = pd.to_datetime(fecha_accidente, format="%Y/%m/")

In [21]:
"""
Resultados finales
"""

vib_ripte = cliente['actualizados'].mean().round(2)

fecha_calculo_final = cliente['fechas'].max()

tasa_interes = tasas[tasas['fechas'] > fecha_calculo_final]['tasas'].sum()/100 + 1

vib_ripte_tasa = (vib_ripte * tasa_interes).round(2)

tasa_interes = str(((tasa_interes - 1) * 100).round(2)) + " %"

indemnizacion = 53 * vib_ripte_tasa *  (65/edad) * incapacidad/(100)
indemnizacion = round(indemnizacion,2)

if tipo_accidente == 1:
    indemnizacion = round(indemnizacion*1.2,2)
else:
    indemnizacion = indemnizacion


In [22]:
print(vib_ripte)
print(tasa_interes)
print(vib_ripte_tasa)
print(indemnizacion)

75814.7
94.48 %
147445.79
7183874.84
